In [2]:
import time
import os
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from sentence_transformers.util import cos_sim
import sys
import key_words
from IPython.display import clear_output

import pickle
def save_obj(obj:object,name:str):
    ext = '.pickle'
    with open(name + ext, 'wb') as handle:
        pickle.dump(obj, handle, protocol=pickle.HIGHEST_PROTOCOL)

def load_obj(name:str)->object:
    ext = '.pickle'
    with open(name + ext, 'rb') as handle:
        return pickle.load(handle)

In [3]:
train_path = "data/train_phase1.tsv"
df = pd.read_csv(train_path, sep="\t")

In [54]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

In [4]:
model_answers = load_obj('data/essaySet_3_model_answers')
docs = df.query(f'EssaySet == {3}')["EssayText"].values.tolist()

In [5]:
enclosure_hard = "\"\""
enclosure_soft = "\"\"@@"
enclosure_mediam = "\"\"@"

In [6]:
answer = "hazem is playing \"\"@football@\"\" in the \"\"@@stadium@@\"\""
answer

'hazem is playing ""@football@"" in the ""@@stadium@@""'

In [33]:
set(['@football@', '@@stadium@@'])-set(['@football@'])

{'@@stadium@@'}

In [7]:
manual_keywords_soft = key_words.get_str_between(answer, enclosure_soft)
manual_keywords_mediam = key_words.get_str_between(answer, enclosure_mediam)
manual_keywords_hard = key_words.get_str_between(answer, enclosure_hard)

In [12]:
manual_keywords_soft, manual_keywords_mediam, manual_keywords_hard

(['stadium'], ['football', '@stadium@'], ['@football@', '@@stadium@@'])

In [27]:
for i in manual_keywords_mediam:
    answer = answer.replace(enclosure_mediam+str(i)+enclosure_mediam[::-1], '')

In [28]:
answer

'hazem is playing  in the '

In [1]:
def get_candidates(n_grams, doc):
    x = list(map(lambda gram :
        key_words.candidates_tokens(str(doc), n_gram_range=gram)
        , n_grams))
    return x

In [48]:
def flatten(ls_ls):
    out = []
    for ls in ls_ls:
        if isinstance(ls, list):
            out.extend(ls)
        else:
            out.append(ls)
    return out

In [55]:
words = set(manual_keywords_soft) - set(manual_keywords_mediam)
ngrams = [*set(list(map(lambda x : len(x.split(' ')),  list(words))))]
ngram = max(ngrams), min(ngrams)
docs_candidates = list(map(lambda doc : get_candidates([ngram], doc), docs))
# flatten
docs_candidates = list(map(flatten , docs_candidates))
docs_candidates_emb = list(map(model.encode, docs_candidates))

In [59]:
np.array(docs_candidates_emb).shape

C:\Users\oeb\AppData\Local\Temp/ipykernel_2248/951429711.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array(docs_candidates_emb).shape


(1891,)

In [67]:
soft_threshold = 0.75
mediam_threshold = 0.88

In [69]:
x = np.array(list(map(lambda docs_cand :cos_sim(model.encode(manual_keywords_soft), docs_cand).__array__().max(axis=1) > soft_threshold, docs_candidates_emb))).astype(float)

In [75]:
y = np.array(list(map(lambda docs_cand :cos_sim(model.encode(manual_keywords_mediam), docs_cand).__array__().max(axis=1) > mediam_threshold, docs_candidates_emb[:10]))).astype(float)